In [1]:
import cv2
import ipywidgets.widgets as widgets
import time

In [2]:
# 设置画帧显示参数
frame_width = 640
frame_height = 360
FPS = 30

In [3]:
# 初始化显示控件
image_widget = widgets.Image(format='jpeg', width=frame_width, height=frame_height)
display(image_widget)

Image(value=b'', format='jpeg', height='360', width='640')

In [4]:
# 格式转换函数
import enum
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [5]:
# 初始化摄像头
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_width)       
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,frame_height)
cap.set(cv2.CAP_PROP_FPS, FPS)  #设置帧率
ret, frame = cap.read()
image_widget.value = bgr8_to_jpeg(frame)

In [6]:
import numpy as np
def detect_and_mark_color_objects(image, color_type):
    # 定义颜色范围（根据你指定的颜色类型）
    color_ranges = {
        "red": ([0, 50, 50], [10, 255, 255]),  # 红色范围（HSV）
        "green": ([40, 50, 50], [80, 255, 255]),  # 绿色范围（HSV）
        "blue": ([100, 50, 50], [140, 255, 255])  # 蓝色范围（HSV）
    }

    # 读取图像
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # 根据颜色类型获取颜色范围
    lower_color = np.array(color_ranges[color_type][0])
    upper_color = np.array(color_ranges[color_type][1])

    # 创建颜色掩膜
    mask = cv2.inRange(hsv, lower_color, upper_color)

    # 使用形态学操作去噪点
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))

    # 查找颜色区域的轮廓
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) > 500:  # 忽略小的轮廓区域
            # 获取轮廓的边界框
            x, y, w, h = cv2.boundingRect(contour)

            # 在原图上画出白色矩形框
            cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 255), 2)

            # 计算物体的中心点
            center_x = x + w // 2
            center_y = y + h // 2

            # 在中心点处画白色十字标记
            cv2.drawMarker(image, (center_x, center_y), (255, 255, 255), markerType=cv2.MARKER_CROSS, markerSize=20,
                           thickness=2)

    return image


In [7]:
while 1:
    ret, frame = cap.read()
    # 调用函数，传入图片路径和颜色类型
    frame = detect_and_mark_color_objects(frame, "red")  # 颜色类型可以是 "red", "green", "blue"
    image_widget.value = bgr8_to_jpeg(frame)
    time.sleep(0.010)

KeyboardInterrupt: 

In [8]:
# 释放摄像头资源，多进程控制摄像头必须运行！
cap.release()